In [ ]:
# | default_exp wb_clim

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [ ]:
# | export
import collections
from typing import Dict
from pandera.typing import DataFrame
from sureau_ecos_py.climate_utils import compute_vpd_from_t_rh

from sureau_ecos_py.create_simulation_parameters import (
    create_simulation_parameters,
)

from sureau_ecos_py.create_climate_data import create_climate_data
from sureau_ecos_py.create_modeling_options import create_modeling_options



In [ ]:
# | export

def new_wb_clim(
    climate_data:DataFrame, # Dataframe created using the `create_climate_data` function
    year:int, # Year, __No definition found__
    day_of_year:int # Day of the year, __No definition found__


) -> Dict:  # Dictionary containing parameters to run the model
    "Create a list with all necessary daily climate values to run SureauR from climate_data"

    # Assert parameters ---------------------------------------------------------

    # Year
    assert isinstance(
        year, int
    ) and 3000 >= year > 0,"year must be a integer value between 0-3000"

    # Day of year
    assert isinstance(
        day_of_year, int
    ) and 366 >= day_of_year >= 1, "day_of_year must be a integer value between 1-365"

    # Create wb_clim dictionary -------------------------------------------------

    # Check that year and day_of_year are present inside the dataframe
    if year in climate_data['year'].values and day_of_year in climate_data['day_of_year'].values:

        # Get row index in climate frame based on year and doy
        row_index = climate_data[(climate_data['year'] == year) & (climate_data['day_of_year'] == day_of_year)].index[0]

        # Transfrom row to a dictionary with params
        wb_clim_dict = collections.defaultdict(list, dict(climate_data.loc[row_index]))

    else:
        raise ValueError(
            f"year:{year} and/or day_of_year:{day_of_year} not found in climate dataframe"
        )

    # Add parameters to dictionary ----------------------------------------------
    wb_clim_dict['net_radiation'] = float("NAN")
    wb_clim_dict['etp'] = float("NAN")
    wb_clim_dict['vpd'] = compute_vpd_from_t_rh(relative_humidity = wb_clim_dict['RHair_mean'],
                                                temperature= wb_clim_dict['Tair_mean']
                                                )

    # Add Temperature from previous and next days

    # cas normal

    # if the row_index is not the first nor the last
    if row_index != 0 and row_index != climate_data.shape[0]:
        wb_clim_dict['Tair_min_prev'] = climate_data.loc[row_index - 1]['Tair_min']
        wb_clim_dict['Tair_min_next'] = climate_data.loc[row_index + 1]['Tair_min']
        wb_clim_dict['Tair_max_prev'] = climate_data.loc[row_index - 1]['Tair_max']

        return wb_clim_dict

    # si premier jour de le la simu

    # if the row_index is the first
    elif row_index == 0:
        print('Firts day of the simulation, previous Tair is the same as the current')

        wb_clim_dict['Tair_min_prev'] = climate_data.loc[row_index]['Tair_min']
        wb_clim_dict['Tair_min_next'] = climate_data.loc[row_index + 1]['Tair_min']
        wb_clim_dict['Tair_max_prev'] = climate_data.loc[row_index]['Tair_max']

        return wb_clim_dict

    elif row_index == climate_data.shape[0]:
        print('Last day of the simulation, next Tair_min_next is the same as the Tair_min')

        wb_clim_dict['Tair_min_prev'] = climate_data.loc[row_index - 1]['Tair_min']
        wb_clim_dict['Tair_min_next'] = climate_data.loc[row_index]['Tair_min']
        wb_clim_dict['Tair_max_prev'] = climate_data.loc[row_index - 1]['Tair_max']

        return wb_clim_dict

    else:
        raise ValueError(
            "Error setting previous and following temperature conditions"
        )


#### __Example: Create new_wb_clim dictionary__

In [ ]:
simulation_parameters_dict = create_simulation_parameters(
    main_dir="./sample_data",
    start_year_simulation=1991,
    end_year_simulation=1991,
    output_type=None,
    output_path="./sample_data",
    overwrite=True,
    resolution_output="subdaily",
)

In [ ]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="turgor",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="vg",
)

In [ ]:
climate_dataframe = create_climate_data(
    modeling_options=modeling_options_dict,
    simulation_parameters=simulation_parameters_dict,
    file_path="./sample_data/climat_example.csv",
    sep=";",
)
climate_dataframe

365 days were selected in the input climate file, covering the period: 1991 - 1991


,DATE,Tair_min,Tair_max,Tair_mean,RG_sum,RHair_min,RHair_max,RHair_mean,WS_mean,PPT_sum,day_of_year,day_of_month,month,year
1,1991-01-01,2.6,7.3,4.95,3.20,95.160421,95.160421,95.160421,3.1,1.2,1,1,1,1991
2,1991-01-02,3.5,9.0,6.25,0.76,90.403122,95.000000,92.701561,5.8,9.8,2,2,1,1991
3,1991-01-03,8.5,10.7,9.60,1.16,92.731052,95.000000,93.865526,4.8,4.8,3,3,1,1991
4,1991-01-04,3.4,9.6,6.50,1.16,96.644009,96.644009,96.644009,2.6,5.0,4,4,1,1991
5,1991-01-05,2.3,6.3,4.30,3.25,96.915145,96.915145,96.915145,5.4,1.0,5,5,1,1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,1991-12-27,4.1,6.6,5.35,2.23,85.326980,95.000000,90.163490,1.9,0.2,361,27,12,1991
362,1991-12-28,-1.5,5.9,2.20,4.19,96.915145,96.915145,96.915145,1.4,0.0,362,28,12,1991
363,1991-12-29,-2.6,4.0,0.70,4.72,96.915145,96.915145,96.915145,1.4,0.0,363,29,12,1991
364,1991-12-30,2.4,4.4,3.40,1.19,96.915145,96.915145,96.915145,1.8,0.0,364,30,12,1991


In [ ]:
new_wb_clim(
    climate_data = climate_dataframe,
    year = 1991,
    day_of_year = 365
)

Last day of the simulation, next Tair_min_next is the same as the Tair_min


defaultdict(list,
            {'DATE': Timestamp('1991-12-31 00:00:00'),
             'Tair_min': -0.2,
             'Tair_max': 2.7,
             'Tair_mean': 1.25,
             'RG_sum': 0.71,
             'RHair_min': 96.9151449,
             'RHair_max': 96.9151449,
             'RHair_mean': 96.9151449,
             'WS_mean': 1.7,
             'PPT_sum': 0.0,
             'day_of_year': 365,
             'day_of_month': 31,
             'month': 12,
             'year': 1991,
             'net_radiation': nan,
             'etp': nan,
             'vpd': 0.020627738121622427,
             'Tair_min_prev': 2.4,
             'Tair_min_next': -0.2,
             'Tair_max_prev': 4.4})